In [35]:
import pandas as pd

# Specify the file path
file_path = 'C:/Users/Youcode/OneDrive/Bureau/workspace/BI/airflow/csv/df_final2.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
print(df)


           itemId                                              title  \
0    1.005006e+15  Fast Charging Cable FM Transmitter Digital Dis...   
1    1.005006e+15  Apple iPhone 13 Pro max 128GB/256GB A15 Bionic...   
2    1.005006e+15  Apple iPhone 12 Unlocked Cellphone 6.1" Screen...   
3    1.005006e+15  Apple iPhone 14 Pro 128GB/256GB 1SIM+1eSIM Unl...   
4    1.005006e+15  Miniso Spiderman Phone Case for iPhone 15 14 1...   
..            ...                                                ...   
795  1.005006e+15  6 In 1 3Pcs Anti Peeping Tempered Glass+3Pcs C...   
796  1.005006e+15  Bass Fish Fisherman Phone Case For iphone 15 1...   
797  1.005006e+15  2-in-1 Tempered Glass For iPhone 15 Pro Camera...   
798  1.005006e+15  UV Glue Tempered Glass For iphone 13 14 15 Pro...   
799  1.005006e+15  2Pcs Privacy Full Cover Screen Protector For I...   

    available        catId sales  wishCount  \
0        True     410204.0     0        0.0   
1        True    5090301.0     0        1

In [11]:
df.columns

Index(['itemId', 'title', 'available', 'catId', 'sales', 'wishCount',
       'itemUrl', 'images', 'properties_list', 'description_images',
       'sku_quantity', 'sku_price', 'sku_promotionPrice', 'sku_unit',
       'sku_isBulk', 'sku_bulk_quantity', 'sku_bulk_discount', 'reviews_count',
       'reviews_averageStarRate', 'shippingFrom', 'shippingTo',
       'shippingOutDays', 'packageDetail', 'service_title', 'service_desc',
       'seller_sellerId', 'seller_storeTitle', 'seller_storeId',
       'seller_companyId', 'seller_storeRating'],
      dtype='object')

In [36]:
import ast
import json

# # Transformation 1: Convert 'itemId'and 'catId' to string
# df['itemId'] = df['itemId'].astype(str)
# df['seller_storeId'] = df['seller_storeId'].astype(str)

df['wishCount'] = df['wishCount'].fillna(0).astype('float')
df['reviews_count'] = df['reviews_count'].fillna(0).astype('float')
df['reviews_averageStarRate'] = df['reviews_averageStarRate'].fillna(0).astype('float')

# Assuming 'sku_price' is a range (e.g., '7.92 - 16.23'), we can create two new columns 'min_price' and 'max_price'
df[['min_price', 'max_price']] = df['sku_price'].str.split(' - ', expand=True)
df[['min_promotionPrice', 'max_promotionPrice']] = df['sku_promotionPrice'].str.split(' - ', expand=True)

df['min_price'] = pd.to_numeric(df['min_price'], errors='coerce')
df['max_price'] = pd.to_numeric(df['max_price'], errors='coerce')

df['min_promotionPrice'] = pd.to_numeric(df['min_promotionPrice'], errors='coerce')
df['max_promotionPrice'] = pd.to_numeric(df['max_promotionPrice'], errors='coerce')

df['min_price'] = df['min_price'].fillna(0).astype('float')
df['max_price'] = df['max_price'].fillna(0).astype('float')

df['min_promotionPrice'] = df['min_promotionPrice'].fillna(0).astype('float')
df['max_promotionPrice'] = df['max_promotionPrice'].fillna(0).astype('float')

df['sku_quantity'] = df['sku_quantity'].fillna(0).astype('float')
df['sku_bulk_quantity'] = df['sku_bulk_discount'].fillna(0).astype('float')
df['sku_bulk_discount'] = df['sku_bulk_discount'].fillna(0).astype('float')
df['shippingOutDays'] = df['shippingOutDays'].fillna(0).astype('float')

# Remove non-numeric characters from 'sales' column
df['sales'] = df['sales'].str.replace(r'[^0-9]', '', regex=True)

# Convert 'sales' column to float, handling empty strings gracefully
df['sales'] = pd.to_numeric(df['sales'], errors='coerce')

# Fill NaN values with 0
df['sales'] = df['sales'].fillna(0).astype('float')



# Extract values from the 'packageDetail' dictionary into separate columns
package_detail_df = pd.DataFrame(df['packageDetail'].apply(lambda x: {} if pd.isna(x) else eval(x)).tolist(), index=df.index)
df[['package_weight', 'package_length', 'package_height', 'package_width']] = package_detail_df[['weight', 'length', 'height', 'width']]

df['service_title'] = df['service_title'].str.replace('[\[\]]', '', regex=True)

df['service_desc'] = df['service_desc'].str.replace('[\[\]]', '', regex=True)

df = df.drop(['itemUrl', 'images', 'description_images', 'catId', 'packageDetail', 'properties_list', 'sku_price', 'sku_promotionPrice', 'sku_unit', 'sku_promotionPrice', 'seller_companyId'], axis=1)

df = df.dropna(subset=['shippingFrom'])
df = df.dropna(subset=['shippingTo'])
df = df.dropna(subset=['sku_price'])
df = df.dropna(subset=['sku_promotionPrice'])

#shipping

# # Assuming df is your main DataFrame
# shipping_columns = ['shippingFrom', 'shippingTo']
# shipping_dim = df[shipping_columns].drop_duplicates()

# # Reset the index to generate a new shipping_id
# shipping_dim.reset_index(drop=True, inplace=True)
# shipping_dim['shipping_id'] = shipping_dim.index + 1  # Assuming 1-based index

# #sercive

# # Assuming df is your main DataFrame
# service_columns = ['service_title', 'service_desc']
# service_dim = df[service_columns].drop_duplicates()

# # Reset the index to generate a new shipping_id
# service_dim.reset_index(drop=True, inplace=True)
# service_dim['service_id'] = service_dim.index + 1  # Assuming 1-based index


# # Merge shipping_dim with the main DataFrame df
# df = pd.merge(df, shipping_dim, on=['shippingFrom', 'shippingTo'], how='left')
# df = pd.merge(df, service_dim, on=['service_title', 'service_desc'], how='left')



df


,itemId,title,available,sales,wishCount,sku_quantity,sku_isBulk,sku_bulk_quantity,sku_bulk_discount,reviews_count,...,shipping_id,service_id,min_price,max_price,min_promotionPrice,max_promotionPrice,package_weight,package_length,package_height,package_width
0,1.005006e+15,Fast Charging Cable FM Transmitter Digital Dis...,True,0.0,0.0,6000.0,True,3.0,3.0,0.0,...,1,1,7.92,16.23,3.72,16.23,0.073,10.0,4.0,6.0
1,1.005006e+15,Apple iPhone 13 Pro max 128GB/256GB A15 Bionic...,True,0.0,1.0,600.0,False,0.0,0.0,0.0,...,1,2,954.66,1060.58,668.26,1060.58,0.500,21.0,9.0,11.0
2,1.005006e+15,"Apple iPhone 12 Unlocked Cellphone 6.1"" Screen...",True,0.0,0.0,2400.0,False,0.0,0.0,0.0,...,1,2,635.15,716.29,317.58,716.29,0.430,24.0,7.0,17.0
3,1.005006e+15,Apple iPhone 14 Pro 128GB/256GB 1SIM+1eSIM Unl...,True,0.0,1.0,480.0,False,0.0,0.0,0.0,...,1,2,1169.36,1315.35,818.55,1315.35,0.500,21.0,9.0,11.0
4,1.005006e+15,Miniso Spiderman Phone Case for iPhone 15 14 1...,True,1.0,1.0,959.0,False,0.0,0.0,0.0,...,1,1,2.15,0.00,0.99,0.00,0.050,20.0,10.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,1.005006e+15,6 In 1 3Pcs Anti Peeping Tempered Glass+3Pcs C...,True,0.0,0.0,600000.0,True,5.0,5.0,0.0,...,1,1,4.69,7.89,2.34,7.89,0.063,15.0,5.0,10.0
796,1.005006e+15,Bass Fish Fisherman Phone Case For iphone 15 1...,True,0.0,0.0,12750.0,True,10.0,10.0,0.0,...,1,2,2.99,0.00,1.88,0.00,0.030,12.0,2.0,18.0
797,1.005006e+15,2-in-1 Tempered Glass For iPhone 15 Pro Camera...,True,0.0,0.0,40016.0,True,3.0,3.0,0.0,...,1,1,1.86,2.35,1.49,2.35,0.100,10.0,10.0,10.0
798,1.005006e+15,UV Glue Tempered Glass For iphone 13 14 15 Pro...,True,0.0,0.0,99456.0,True,3.0,3.0,0.0,...,1,2,3.51,7.74,1.76,7.74,0.010,1.0,1.0,1.0


In [228]:
# Assuming 'df' is your DataFrame
df.to_csv('C:/Users/Youcode/OneDrive/Bureau/workspace/BI/airflow/csv/df_final1.csv', index=False)


In [37]:
product_columns = ['itemId', 'title', 'available', 'sku_isBulk']
product_dim = df[product_columns]
product_dim = product_dim.drop_duplicates()

product_dim

,itemId,title,available,sku_isBulk
0,1.005006e+15,Fast Charging Cable FM Transmitter Digital Dis...,True,True
1,1.005006e+15,Apple iPhone 13 Pro max 128GB/256GB A15 Bionic...,True,False
2,1.005006e+15,"Apple iPhone 12 Unlocked Cellphone 6.1"" Screen...",True,False
3,1.005006e+15,Apple iPhone 14 Pro 128GB/256GB 1SIM+1eSIM Unl...,True,False
4,1.005006e+15,Miniso Spiderman Phone Case for iPhone 15 14 1...,True,False
...,...,...,...,...
794,1.005006e+15,2Pcs Privacy Protective Glass For iPhone 14 Pr...,True,True
796,1.005006e+15,Bass Fish Fisherman Phone Case For iphone 15 1...,True,True
797,1.005006e+15,2-in-1 Tempered Glass For iPhone 15 Pro Camera...,True,True
798,1.005006e+15,UV Glue Tempered Glass For iphone 13 14 15 Pro...,True,True


In [38]:
shipping_columns = ['shipping_id', 'shippingFrom', 'shippingTo']
shipping_dim = df[shipping_columns]
shipping_dim = shipping_dim.drop_duplicates()

shipping_dim


,shipping_id,shippingFrom,shippingTo
0,1,China,United States
142,3,United States,United States


In [39]:
service_columns = ['service_id', 'service_title', 'service_desc']
service_dim = df[service_columns]
service_dim = service_dim.drop_duplicates()

service_dim

,service_id,service_title,service_desc
0,1,"'Free returns', 'On-time guarantee'",'Not satisfied with your order? Send it back w...
1,2,'Free returns','Not satisfied with your order? Send it back w...


In [40]:
seller_columns = ['seller_sellerId', 'seller_storeTitle']
seller_dim = df[seller_columns]
seller_dim = seller_dim.drop_duplicates()

seller_dim

,seller_sellerId,seller_storeTitle
0,2.330540e+08,Simphone Store
1,2.674635e+09,HS Phone Store
2,2.677514e+09,IBuy Online Shopping - The Genuine Mobile Phon...
4,2.299728e+08,Minxin Factory Store
5,2.201247e+08,Wolfrule Official Store
...,...,...
786,2.538832e+08,dcedg Store
787,2.404224e+08,3CC Digital Global Store
788,2.678067e+09,GM Glass Store
789,1.179454e+08,PHONECOVER Store


In [41]:
fact_table_columns = ['itemId', 'shipping_id', 'service_id', 'seller_sellerId', 'sales','sku_quantity', 'wishCount', 'reviews_count', 'reviews_averageStarRate', 'sku_bulk_quantity', 'sku_bulk_discount', 'min_price', 'max_price', 'min_promotionPrice', 'max_promotionPrice', 'shippingOutDays', 'package_weight', 'package_length', 'package_height', 'package_width', 'seller_storeRating']
fact_table = df[fact_table_columns]
fact_table = fact_table.drop_duplicates()

fact_table


,itemId,shipping_id,service_id,seller_sellerId,sales,sku_quantity,wishCount,reviews_count,reviews_averageStarRate,sku_bulk_quantity,...,min_price,max_price,min_promotionPrice,max_promotionPrice,shippingOutDays,package_weight,package_length,package_height,package_width,seller_storeRating
0,1.005006e+15,1,1,2.330540e+08,0.0,6000.0,0.0,0.0,0.0,3.0,...,7.92,16.23,3.72,16.23,7.0,0.073,10.0,4.0,6.0,89.4
1,1.005006e+15,1,2,2.674635e+09,0.0,600.0,1.0,0.0,0.0,0.0,...,954.66,1060.58,668.26,1060.58,5.0,0.500,21.0,9.0,11.0,100.0
2,1.005006e+15,1,2,2.677514e+09,0.0,2400.0,0.0,0.0,0.0,0.0,...,635.15,716.29,317.58,716.29,5.0,0.430,24.0,7.0,17.0,0.0
3,1.005006e+15,1,2,2.674635e+09,0.0,480.0,1.0,0.0,0.0,0.0,...,1169.36,1315.35,818.55,1315.35,5.0,0.500,21.0,9.0,11.0,100.0
4,1.005006e+15,1,1,2.299728e+08,1.0,959.0,1.0,0.0,0.0,0.0,...,2.15,0.00,0.99,0.00,7.0,0.050,20.0,10.0,10.0,94.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,1.005006e+15,1,1,2.542121e+08,0.0,75000.0,0.0,0.0,0.0,10.0,...,2.09,6.03,1.04,6.03,7.0,0.038,15.0,5.0,10.0,81.5
796,1.005006e+15,1,2,2.523678e+08,0.0,12750.0,0.0,0.0,0.0,10.0,...,2.99,0.00,1.88,0.00,5.0,0.030,12.0,2.0,18.0,91.4
797,1.005006e+15,1,1,2.318936e+08,0.0,40016.0,0.0,0.0,0.0,3.0,...,1.86,2.35,1.49,2.35,7.0,0.100,10.0,10.0,10.0,88.5
798,1.005006e+15,1,2,2.677634e+09,0.0,99456.0,0.0,0.0,0.0,3.0,...,3.51,7.74,1.76,7.74,5.0,0.010,1.0,1.0,1.0,100.0


In [236]:
from sqlalchemy import create_engine, select, insert, update, MetaData, Table
import pandas as pd

# Connection parameters
server_name = 'localhost'
database_name = 'warehouse'
user_id = 'amine'
password = 'Amine'

# Create a connection string
connection_str = f'mssql+pyodbc://{user_id}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server'

# Create an SQLAlchemy engine
engine = create_engine(connection_str)

# Create a table object for 'ProductDim'
metadata = MetaData()
product_dim_table = Table('ProductDim', metadata, autoload_with=engine)

# Create a table object for 'ShippingDim'
shipping_dim_table = Table('ShippingDim', metadata, autoload_with=engine)

# Create a table object for 'ServiceDim'
service_dim_table = Table('ServiceDim', metadata, autoload_with=engine)

# Create a table object for 'SellerDim'
seller_dim_table = Table('SellerDim', metadata, autoload_with=engine)


def insert_or_update_product(product_dim):
    with engine.connect() as connection:
        for _, row in product_dim.iterrows():
            item_id = row['itemId']
            title = row['title']
            available = row['available']
            sku_is_bulk = row['sku_isBulk']

            # Check if the 'itemId' already exists
            existing_row = connection.execute(select([product_dim_table]).where(product_dim_table.c.itemId == item_id)).fetchone()

            if existing_row:
                # 'itemId' already exists, perform update
                update_stmt = (
                    update(product_dim_table)
                    .values(title=title, available=available, sku_isBulk=sku_is_bulk)
                    .where(product_dim_table.c.itemId == item_id)
                )
                connection.execute(update_stmt)
            else:
                # 'itemId' does not exist, perform insert
                insert_stmt = insert(product_dim_table).values(
                    itemId=item_id,
                    title=title,
                    available=available,
                    sku_isBulk=sku_is_bulk
                )
                connection.execute(insert_stmt)

def insert_or_update_shipping(shipping_dim):
    with engine.connect() as connection:
        for _, row in shipping_dim.iterrows():
            shipping_id = row['shipping_id']
            shipping_from = row['shippingFrom']
            shipping_to = row['shippingTo']

            # Check if the 'shipping_id' already exists
            existing_row = connection.execute(select([shipping_dim_table]).where(shipping_dim_table.c.shipping_id == shipping_id)).fetchone()

            if existing_row:
                # 'shipping_id' already exists, perform update
                update_stmt = (
                    update(shipping_dim_table)
                    .values(shippingFrom=shipping_from, shippingTo=shipping_to)
                    .where(shipping_dim_table.c.shipping_id == shipping_id)
                )
                connection.execute(update_stmt)
            else:
                # 'shipping_id' does not exist, perform insert
                insert_stmt = insert(shipping_dim_table).values(
                    shipping_id=shipping_id,
                    shippingFrom=shipping_from,
                    shippingTo=shipping_to
                )
                connection.execute(insert_stmt)

def insert_or_update_service(service_dim):
    with engine.connect() as connection:
        for _, row in service_dim.iterrows():
            service_id = row['service_id']
            service_title = row['service_title']
            service_desc = row['service_desc']

            # Check if the 'service_id' already exists
            existing_row = connection.execute(select([service_dim_table]).where(service_dim_table.c.service_id == service_id)).fetchone()

            if existing_row:
                # 'service_id' already exists, perform update
                update_stmt = (
                    update(service_dim_table)
                    .values(service_title=service_title, service_desc=service_desc)
                    .where(service_dim_table.c.service_id == service_id)
                )
                connection.execute(update_stmt)
            else:
                # 'service_id' does not exist, perform insert
                insert_stmt = insert(service_dim_table).values(
                    service_id=service_id,
                    service_title=service_title,
                    service_desc=service_desc
                )
                connection.execute(insert_stmt)

def insert_or_update_seller(seller_dim):
    with engine.connect() as connection:
        for _, row in seller_dim.iterrows():
            seller_id = row['seller_sellerId']
            store_title = row['seller_storeTitle']

            # Check if the 'seller_id' already exists
            existing_row = connection.execute(select([seller_dim_table]).where(seller_dim_table.c.seller_sellerId == seller_id)).fetchone()

            if existing_row:
                # 'seller_id' already exists, perform update
                update_stmt = (
                    update(seller_dim_table)
                    .values(seller_storeTitle=store_title)
                    .where(seller_dim_table.c.seller_sellerId == seller_id)
                )
                connection.execute(update_stmt)
            else:
                # 'seller_id' does not exist, perform insert
                insert_stmt = insert(seller_dim_table).values(
                    seller_sellerId=seller_id,
                    seller_storeTitle=store_title
                )
                connection.execute(insert_stmt)


# Call the functions
insert_or_update_product(product_dim)
insert_or_update_shipping(shipping_dim)
insert_or_update_service(service_dim)
insert_or_update_seller(seller_dim)

fact_table.to_sql('SalesFact', engine, if_exists='replace', index=False)



-1

In [34]:
# Specify the output file path for the updated DataFrame
output_file_path = 'C:/Users/Youcode/OneDrive/Bureau/workspace/BI/airflow/csv/df_final2.csv'

# Save the DataFrame to a new CSV file
df.to_csv(output_file_path, index=False)